In [1]:
import numpy as np

class Vector3:
    """A simple 3D vector class for position, velocity, and acceleration."""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    def __str__(self):
        return f"({self.x:.2f}, {self.y:.2f}, {self.z:.2f})"


class Entity:
    """An entity in the physics simulation with physical properties."""
    def __init__(self, x=0.0, y=10.0, z=0.0, mass=1.0):
        self.position = Vector3(x, y, z)
        self.velocity = Vector3(0, 0, 0)
        self.acceleration = Vector3(0, 0, 0)
        self.mass = mass
        self.restitution = 0.7  # Bounciness factor (0 = no bounce, 1 = perfect bounce)
        self.grounded = False


class PhysicsSimulation:
    """Physics simulation with gravity and basic collision handling."""
    def __init__(self):
        # Earth's gravity in m/s²
        self.gravity = 9.81
        
        # Simulation settings
        self.time_step = 0.016  # 60fps equivalent in seconds
        self.ground_level = 0.0  # y-coordinate of ground level
        
        # Collection of all entities in the simulation
        self.entities = []
    
    def add_entity(self, entity):
        """Add an entity to the simulation."""
        # Ensure entity is at or above ground level
        if entity.position.y < self.ground_level:
            entity.position.y = self.ground_level
        
        self.entities.append(entity)
        return entity
    
    def create_entity(self, x=0.0, y=10.0, z=0.0, mass=1.0):
        """Create a new entity with physics properties."""
        entity = Entity(x, y, z, mass)
        return self.add_entity(entity)
    
    def apply_force(self, entity, force_x, force_y, force_z):
        """Apply a force to an entity."""
        # F = ma, so a = F/m
        entity.acceleration.x += force_x / entity.mass
        entity.acceleration.y += force_y / entity.mass
        entity.acceleration.z += force_z / entity.mass
        
        # Apply acceleration to velocity
        entity.velocity.x += entity.acceleration.x * self.time_step
        entity.velocity.y += entity.acceleration.y * self.time_step
        entity.velocity.z += entity.acceleration.z * self.time_step
        
        # Reset acceleration
        entity.acceleration.x = 0
        entity.acceleration.y = 0
        entity.acceleration.z = 0
    
    def is_grounded(self, entity):
        """Check if an entity is on the ground."""
        return abs(entity.position.y - self.ground_level) < 0.01 and entity.velocity.y <= 0
    
    def update(self):
        """Apply gravitational force to all entities and update their positions."""
        for entity in self.entities:
            # Only apply gravity if not grounded or if about to leave the ground (positive velocity)
            if not entity.grounded or entity.velocity.y > 0:
                entity.velocity.y -= self.gravity * self.time_step
            
            # Update position based on velocity
            entity.position.x += entity.velocity.x * self.time_step
            entity.position.y += entity.velocity.y * self.time_step
            entity.position.z += entity.velocity.z * self.time_step
            
            # Check for ground collision
            if entity.position.y <= self.ground_level:
                entity.position.y = self.ground_level
                entity.grounded = True
                
                # Bounce if moving downward
                if entity.velocity.y < 0:
                    entity.velocity.y = -entity.velocity.y * entity.restitution
                    
                    # Apply friction to horizontal movement
                    entity.velocity.x *= 0.95
                    entity.velocity.z *= 0.95
                    
                    # Stop very small bounces
                    if abs(entity.velocity.y) < 0.1:
                        entity.velocity.y = 0
            else:
                entity.grounded = False
            



def run_simulation(steps=100, step_size=10):
    """Run the simulation for a specified number of steps."""
    # Create the simulation
    simulation = PhysicsSimulation()
    
    # Create an entity (starting at x=0, y=10, z=0 with mass=1)
    agent = simulation.create_entity()
    
    print(f"Starting simulation with gravity: {simulation.gravity} m/s²")
    print(f"Step 0: Position {agent.position}, "
          f"Velocity: ({agent.velocity.x:.2f}, {agent.velocity.y:.2f}, {agent.velocity.z:.2f}), "
          f"Grounded: {agent.grounded}")
    
    for i in range(1, steps+1):
        simulation.update()
        
        # Log based on specified step size
        if i % step_size == 0:
            print(f"Step {i}: Position {agent.position}, "
                  f"Velocity: ({agent.velocity.x:.2f}, {agent.velocity.y:.2f}, {agent.velocity.z:.2f}), "
                  f"Grounded: {agent.grounded}")
    
    print("Simulation complete")


if __name__ == "__main__":
    run_simulation()

Starting simulation with gravity: 9.81 m/s²
Step 0: Position (0.00, 10.00, 0.00), Velocity: (0.00, 0.00, 0.00), Grounded: False
Step 10: Position (0.00, 9.86, 0.00), Velocity: (0.00, -1.57, 0.00), Grounded: False
Step 20: Position (0.00, 9.47, 0.00), Velocity: (0.00, -3.14, 0.00), Grounded: False
Step 30: Position (0.00, 8.83, 0.00), Velocity: (0.00, -4.71, 0.00), Grounded: False
Step 40: Position (0.00, 7.94, 0.00), Velocity: (0.00, -6.28, 0.00), Grounded: False
Step 50: Position (0.00, 6.80, 0.00), Velocity: (0.00, -7.85, 0.00), Grounded: False
Step 60: Position (0.00, 5.40, 0.00), Velocity: (0.00, -9.42, 0.00), Grounded: False
Step 70: Position (0.00, 3.76, 0.00), Velocity: (0.00, -10.99, 0.00), Grounded: False
Step 80: Position (0.00, 1.86, 0.00), Velocity: (0.00, -12.56, 0.00), Grounded: False
Step 90: Position (0.00, 0.15, 0.00), Velocity: (0.00, 9.62, 0.00), Grounded: False
Step 100: Position (0.00, 1.56, 0.00), Velocity: (0.00, 8.05, 0.00), Grounded: False
Simulation complete


In [2]:
# Assuming the previous code cell with Vector3, Entity, and PhysicsSimulation has been executed

class Agent(Entity):
    """An agent that can move intentionally within the physics simulation."""
    
    def __init__(self, x=0.0, y=0.0, z=0.0, mass=70.0):
        super().__init__(x, y, z, mass)
        
        # Agent properties
        self.max_speed = 5.0  # Maximum horizontal speed in m/s
        self.jump_force = 10.0  # Jump force in m/s (initial velocity)
        self.move_force = 1000.0  # Horizontal movement force
        self.name = "Agent"
        
        # Movement state
        self.move_direction = Vector3(0, 0, 0)  # Normalized direction vector
        self.is_jumping = False
        self.jump_cooldown = 0
        
        # Force grounded state if starting at ground level
        if abs(self.position.y) < 0.01:
            self.position.y = 0.0
            self.grounded = True
        
    def move(self, direction_x, direction_z):
        """Set the movement direction for the agent."""
        # Normalize the direction vector
        magnitude = (direction_x**2 + direction_z**2)**0.5
        if magnitude > 0:
            self.move_direction.x = direction_x / magnitude
            self.move_direction.z = direction_z / magnitude
        else:
            self.move_direction.x = 0
            self.move_direction.z = 0
        
        # Debug info
        print(f"Agent {self.name} movement direction set to ({self.move_direction.x:.2f}, {self.move_direction.z:.2f})")
    
    def jump(self):
        """Make the agent jump if it's on the ground and not in cooldown."""
        if self.grounded and self.jump_cooldown <= 0:
            self.velocity.y = self.jump_force
            self.is_jumping = True
            self.jump_cooldown = 10  # Cooldown frames before can jump again
            # Temporarily set grounded to False immediately to prevent multiple jumps
            self.grounded = False
            print(f"Agent {self.name} jumped with force {self.jump_force}")
            return True
        else:
            # Debug information
            if not self.grounded:
                print(f"Jump failed: Agent {self.name} is not grounded")
            elif self.jump_cooldown > 0:
                print(f"Jump failed: Agent {self.name} is on cooldown ({self.jump_cooldown} steps left)")
            return False
    
    def update(self, simulation):
        """Update agent's state within the simulation context."""
        # Reduce jump cooldown if active
        if self.jump_cooldown > 0:
            self.jump_cooldown -= 1
        
        # Apply movement force if agent is trying to move
        if self.move_direction.x != 0 or self.move_direction.z != 0:
            # Scale force by mass and apply in move direction
            force_x = self.move_direction.x * self.move_force
            force_z = self.move_direction.z * self.move_force
            
            # Apply more force when on ground (better control) than in air
            ground_multiplier = 1.0 if self.grounded else 0.2
            simulation.apply_force(self, 
                                  force_x * ground_multiplier, 
                                  0,  # No vertical force from movement
                                  force_z * ground_multiplier)
        
        # Apply speed limiting
        horizontal_speed = (self.velocity.x**2 + self.velocity.z**2)**0.5
        if horizontal_speed > self.max_speed:
            # Scale down to max speed
            scale = self.max_speed / horizontal_speed
            self.velocity.x *= scale
            self.velocity.z *= scale
        
        # Apply air resistance/drag when not on ground
        if not self.grounded:
            drag_factor = 0.99  # Slight air resistance
            self.velocity.x *= drag_factor
            self.velocity.z *= drag_factor
        # More friction when on ground and not actively moving
        elif abs(self.move_direction.x) < 0.1 and abs(self.move_direction.z) < 0.1:
            # Stronger friction when not trying to move
            ground_friction = 0.85
            self.velocity.x *= ground_friction
            self.velocity.z *= ground_friction
    
    def stop(self):
        """Stop all horizontal movement."""
        self.move_direction.x = 0
        self.move_direction.z = 0
    
    def __str__(self):
        """Return a string representation of the agent."""
        state = "Grounded" if self.grounded else "Airborne"
        return (f"{self.name} at {self.position}, "
                f"Speed: {(self.velocity.x**2 + self.velocity.z**2)**0.5:.2f} m/s, "
                f"State: {state}")


class World:
    """A world containing physics simulation and agents."""
    
    def __init__(self, width=100.0, depth=100.0):
        self.simulation = PhysicsSimulation()
        self.width = width  # X dimension
        self.depth = depth  # Z dimension
        self.agents = []
        
        # World settings
        self.time_step = self.simulation.time_step
        
    def add_agent(self, x=0.0, y=1.0, z=0.0):
        """Add a new agent to the world."""
        # Create the agent with proper initial height
        agent = Agent(x, y, z)
        
        # Add to physics simulation
        self.simulation.add_entity(agent)
        
        # Add to agent list
        self.agents.append(agent)
        return agent
    
    def update(self):
        """Update the world for one time step."""
        # Update all agents
        for agent in self.agents:
            agent.update(self.simulation)
        
        # Update physics
        self.simulation.update()
        
        # Enforce world boundaries for all agents
        for agent in self.agents:
            # X boundaries
            if agent.position.x < 0:
                agent.position.x = 0
                agent.velocity.x = 0
            elif agent.position.x > self.width:
                agent.position.x = self.width
                agent.velocity.x = 0
                
            # Z boundaries
            if agent.position.z < 0:
                agent.position.z = 0
                agent.velocity.z = 0
            elif agent.position.z > self.depth:
                agent.position.z = self.depth
                agent.velocity.z = 0

In [22]:
# Simple test to verify gravity works correctly
import time

# Create a simple entity
entity = Entity(0.0, 10.0, 0.0)  # Starting 10 units above ground
print(f"Created entity at y={entity.position.y} with velocity.y={entity.velocity.y}")

# Create physics simulation
simulation = PhysicsSimulation()
simulation.add_entity(entity)

# Run simulation manually step by step to see exactly what happens
print("\nRunning gravity simulation:")
for i in range(100):
    # Before update
    old_y = entity.position.y
    old_velocity = entity.velocity.y
    
    # Run physics update
    simulation.update()
    
    # Only print every 5 steps to not flood output
    if i % 5 == 0:
        print(f"Step {i}: Position y: {old_y:.2f} → {entity.position.y:.2f}, "
              f"Velocity y: {old_velocity:.2f} → {entity.velocity.y:.2f}, "
              f"Grounded: {entity.grounded}")
    
    # Stop if on ground and not moving
    if entity.grounded and abs(entity.velocity.y) < 0.01:
        print(f"Entity has come to rest on ground at step {i}")
        break

Created entity at y=10.0 with velocity.y=0.0

Running gravity simulation:
Step 0: Position y: 10.00 → 10.00, Velocity y: 0.00 → -0.16, Grounded: False
Step 5: Position y: 9.96 → 9.95, Velocity y: -0.78 → -0.94, Grounded: False
Step 10: Position y: 9.86 → 9.83, Velocity y: -1.57 → -1.73, Grounded: False
Step 15: Position y: 9.70 → 9.66, Velocity y: -2.35 → -2.51, Grounded: False
Step 20: Position y: 9.47 → 9.42, Velocity y: -3.14 → -3.30, Grounded: False
Step 25: Position y: 9.18 → 9.12, Velocity y: -3.92 → -4.08, Grounded: False
Step 30: Position y: 8.83 → 8.75, Velocity y: -4.71 → -4.87, Grounded: False
Step 35: Position y: 8.42 → 8.33, Velocity y: -5.49 → -5.65, Grounded: False
Step 40: Position y: 7.94 → 7.84, Velocity y: -6.28 → -6.44, Grounded: False
Step 45: Position y: 7.40 → 7.29, Velocity y: -7.06 → -7.22, Grounded: False
Step 50: Position y: 6.80 → 6.67, Velocity y: -7.85 → -8.00, Grounded: False
Step 55: Position y: 6.13 → 5.99, Velocity y: -8.63 → -8.79, Grounded: False
Ste

In [3]:
# Looking at the issue with movement, let's make a direct test
# with focus only on horizontal movement

# First, let's fix the agent.move method
def move_fixed(agent, direction_x, direction_z):
    """Corrected movement function"""
    # Normalize the direction vector
    magnitude = (direction_x**2 + direction_z**2)**0.5
    if magnitude > 0:
        # Here's a key fix - storing direction properly
        agent.move_direction = Vector3(
            direction_x / magnitude,
            0,  # No vertical component to movement direction
            direction_z / magnitude
        )
    else:
        agent.move_direction = Vector3(0, 0, 0)
    
    print(f"Agent direction set to: x={agent.move_direction.x:.2f}, z={agent.move_direction.z:.2f}")


# Create a new test world
world = World(width=50.0, depth=50.0)
agent = world.add_agent(x=25.0, y=0.0, z=25.0)
agent.name = "MovementTest"

# Replace the agent's move method with our fixed version
agent.move = lambda dx, dz: move_fixed(agent, dx, dz)

print(f"Testing movement with fixed method")
print(f"Initial position: {agent.position}")

# Try moving north (negative z)
print("\nMoving NORTH (negative z)...")
agent.move(0, -1)

for i in range(20):
    world.update()
    if i % 5 == 0:
        print(f"Step {i}: Position={agent.position}, Speed={agent.velocity.x:.2f},{agent.velocity.z:.2f}")

# Try moving east (positive x)
print("\nMoving EAST (positive x)...")
agent.move(1, 0)

for i in range(20):
    world.update()
    if i % 5 == 0:
        print(f"Step {i}: Position={agent.position}, Speed={agent.velocity.x:.2f},{agent.velocity.z:.2f}")

# Try moving diagonally
print("\nMoving SOUTHEAST (diagonal)...")
agent.move(1, 1)

for i in range(20):
    world.update()
    if i % 5 == 0:
        print(f"Step {i}: Position={agent.position}, Speed={agent.velocity.x:.2f},{agent.velocity.z:.2f}")

print(f"\nFinal position: {agent.position}")

Testing movement with fixed method
Initial position: (25.00, 0.00, 25.00)

Moving NORTH (negative z)...
Agent direction set to: x=0.00, z=-1.00
Step 0: Position=(25.00, 0.00, 25.00), Speed=0.00,-0.23
Step 5: Position=(25.00, 0.00, 24.92), Speed=0.00,-1.37
Step 10: Position=(25.00, 0.00, 24.76), Speed=0.00,-2.51
Step 15: Position=(25.00, 0.00, 24.50), Speed=0.00,-3.66

Moving EAST (positive x)...
Agent direction set to: x=1.00, z=0.00
Step 0: Position=(25.00, 0.00, 24.16), Speed=0.23,-4.57
Step 5: Position=(25.08, 0.00, 23.79), Speed=1.37,-4.57
Step 10: Position=(25.24, 0.00, 23.43), Speed=2.41,-4.38
Step 15: Position=(25.47, 0.00, 23.11), Speed=3.18,-3.86

Moving SOUTHEAST (diagonal)...
Agent direction set to: x=0.71, z=0.71
Step 0: Position=(25.75, 0.00, 22.83), Speed=3.81,-3.24
Step 5: Position=(26.09, 0.00, 22.61), Speed=4.43,-2.32
Step 10: Position=(26.46, 0.00, 22.47), Speed=4.81,-1.35
Step 15: Position=(26.86, 0.00, 22.41), Speed=4.98,-0.44

Final position: (27.18, 0.00, 22.41)


In [4]:
# Completely redefine the physics from scratch to fix the gravity issue

class Vector3:
    """A simple 3D vector class for position, velocity, and acceleration."""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    def __str__(self):
        return f"({self.x:.2f}, {self.y:.2f}, {self.z:.2f})"


class Entity:
    """An entity in the physics simulation with physical properties."""
    def __init__(self, x=0.0, y=10.0, z=0.0, mass=1.0):
        self.position = Vector3(x, y, z)
        self.velocity = Vector3(0, 0, 0)
        self.acceleration = Vector3(0, 0, 0)
        self.mass = mass
        self.restitution = 0.7  # Bounciness factor (0 = no bounce, 1 = perfect bounce)
        self.grounded = False


class PhysicsSimulation:
    """Physics simulation with gravity and basic collision handling."""
    def __init__(self):
        # Earth's gravity in m/s²
        self.gravity = 9.81
        
        # Simulation settings
        self.time_step = 0.016  # 60fps equivalent in seconds
        self.ground_level = 0.0  # y-coordinate of ground level
        
        # Collection of all entities in the simulation
        self.entities = []
    
    def add_entity(self, entity):
        """Add an entity to the simulation."""
        # Ensure entity is at or above ground level
        if entity.position.y < self.ground_level:
            entity.position.y = self.ground_level
            entity.grounded = True
        
        self.entities.append(entity)
        return entity
    
    def apply_force(self, entity, force_x, force_y, force_z):
        """Apply a force to an entity."""
        # F = ma, so a = F/m
        # Add to acceleration
        entity.acceleration.x += force_x / entity.mass
        entity.acceleration.y += force_y / entity.mass
        entity.acceleration.z += force_z / entity.mass
    
    def update(self):
        """Update the physics simulation for one time step."""
        for entity in self.entities:
            # Reset acceleration at the start of each update
            entity.acceleration.x = 0
            entity.acceleration.y = 0
            entity.acceleration.z = 0
            
            # Apply gravity force (only if not grounded)
            if not entity.grounded:
                entity.acceleration.y -= self.gravity
            
            # Apply acceleration to velocity
            entity.velocity.x += entity.acceleration.x * self.time_step
            entity.velocity.y += entity.acceleration.y * self.time_step
            entity.velocity.z += entity.acceleration.z * self.time_step
            
            # Apply velocity to position
            entity.position.x += entity.velocity.x * self.time_step
            entity.position.y += entity.velocity.y * self.time_step
            entity.position.z += entity.velocity.z * self.time_step
            
            # Handle ground collision
            if entity.position.y <= self.ground_level:
                entity.position.y = self.ground_level
                
                # If moving downward, handle bounce or stop
                if entity.velocity.y < 0:
                    bounce_velocity = -entity.velocity.y * entity.restitution
                    
                    # If bounce is significant, bounce; otherwise stop
                    if bounce_velocity > 0.1:
                        entity.velocity.y = bounce_velocity
                    else:
                        entity.velocity.y = 0
                    
                    # Apply horizontal friction on impact
                    entity.velocity.x *= 0.9
                    entity.velocity.z *= 0.9
                
                # Update grounded state
                entity.grounded = True
            elif entity.velocity.y != 0:
                # In the air and moving
                entity.grounded = False


# Test the fixed physics
def test_gravity():
    print("Testing gravity with fixed physics:")
    
    # Create simulation
    simulation = PhysicsSimulation()
    
    # Create test entity 10 units above ground
    entity = Entity(0, 10, 0)
    simulation.add_entity(entity)
    
    # Run simulation
    for i in range(100):
        old_y = entity.position.y
        old_vy = entity.velocity.y
        
        simulation.update()
        
        # Print details every 10 steps
        if i % 10 == 0:
            print(f"Step {i}: y={entity.position.y:.2f} (was {old_y:.2f}), "
                 f"vy={entity.velocity.y:.2f} (was {old_vy:.2f}), "
                 f"Grounded={entity.grounded}")
        
        # Stop if on ground and not moving
        if entity.grounded and abs(entity.velocity.y) < 0.01:
            print(f"Entity reached ground and stopped at step {i}")
            break

# Run the test
test_gravity()

Testing gravity with fixed physics:
Step 0: y=10.00 (was 10.00), vy=-0.16 (was 0.00), Grounded=False
Step 10: y=9.83 (was 9.86), vy=-1.73 (was -1.57), Grounded=False
Step 20: y=9.42 (was 9.47), vy=-3.30 (was -3.14), Grounded=False
Step 30: y=8.75 (was 8.83), vy=-4.87 (was -4.71), Grounded=False
Step 40: y=7.84 (was 7.94), vy=-6.44 (was -6.28), Grounded=False
Step 50: y=6.67 (was 6.80), vy=-8.00 (was -7.85), Grounded=False
Step 60: y=5.25 (was 5.40), vy=-9.57 (was -9.42), Grounded=False
Step 70: y=3.58 (was 3.76), vy=-11.14 (was -10.99), Grounded=False
Step 80: y=1.66 (was 1.86), vy=-12.71 (was -12.56), Grounded=False
Step 90: y=0.31 (was 0.16), vy=9.62 (was 9.78), Grounded=False


In [30]:
class Agent(Entity):
    """An agent that can move intentionally within the physics simulation."""
    
    def __init__(self, x=0.0, y=0.0, z=0.0, mass=70.0):
        super().__init__(x, y, z, mass)
        
        # Agent properties
        self.max_speed = 5.0  # Maximum horizontal speed in m/s
        self.jump_force = 10.0  # Jump force in m/s (initial velocity)
        self.move_force = 1000.0  # Horizontal movement force
        self.name = "Agent"
        
        # Movement state - IMPORTANT FIX: store as separate values, not as Vector3
        self.move_direction_x = 0.0
        self.move_direction_z = 0.0
        self.is_jumping = False
        self.jump_cooldown = 0
        
        # Force grounded state if starting at ground level
        if abs(self.position.y) < 0.01:
            self.position.y = 0.0
            self.grounded = True
        
    def move(self, direction_x, direction_z):
        """Set the movement direction for the agent."""
        # Calculate magnitude
        magnitude = (direction_x**2 + direction_z**2)**0.5
        
        # Normalize and store direction components
        if magnitude > 0:
            self.move_direction_x = direction_x / magnitude
            self.move_direction_z = direction_z / magnitude
        else:
            self.move_direction_x = 0
            self.move_direction_z = 0
        
        print(f"Agent {self.name} movement direction set to ({self.move_direction_x:.2f}, {self.move_direction_z:.2f})")
    
    def jump(self):
        """Make the agent jump if it's on the ground and not in cooldown."""
        if self.grounded and self.jump_cooldown <= 0:
            # Set upward velocity
            self.velocity.y = self.jump_force
            
            # Update state
            self.is_jumping = True
            self.jump_cooldown = 10  # Cooldown frames
            self.grounded = False  # No longer grounded
            
            print(f"Agent {self.name} jumped with force {self.jump_force}")
            return True
        else:
            # Debug information
            if not self.grounded:
                print(f"Jump failed: Agent {self.name} is not grounded")
            elif self.jump_cooldown > 0:
                print(f"Jump failed: Agent {self.name} is on cooldown ({self.jump_cooldown} steps left)")
            return False
    
    def update(self, simulation):
        """Update agent's state within the simulation context."""
        # Reduce jump cooldown if active
        if self.jump_cooldown > 0:
            self.jump_cooldown -= 1
        
        # Apply movement force if agent is trying to move
        if self.move_direction_x != 0 or self.move_direction_z != 0:
            # Calculate force in each direction
            force_x = self.move_direction_x * self.move_force
            force_z = self.move_direction_z * self.move_force
            
            # Apply more force when on ground (better control) than in air
            ground_multiplier = 1.0 if self.grounded else 0.2
            
            # Apply forces through simulation
            simulation.apply_force(self, 
                                  force_x * ground_multiplier, 
                                  0,  # No vertical force from movement
                                  force_z * ground_multiplier)
        
        # Apply speed limiting for horizontal movement only
        horizontal_speed = (self.velocity.x**2 + self.velocity.z**2)**0.5
        if horizontal_speed > self.max_speed:
            # Scale down to max speed
            scale = self.max_speed / horizontal_speed
            self.velocity.x *= scale
            self.velocity.z *= scale
        
        # Apply air resistance/drag when not on ground
        if not self.grounded:
            drag_factor = 0.99  # Slight air resistance
            self.velocity.x *= drag_factor
            self.velocity.z *= drag_factor
        # More friction when on ground and not actively moving
        elif abs(self.move_direction_x) < 0.1 and abs(self.move_direction_z) < 0.1:
            # Stronger friction when not trying to move
            ground_friction = 0.85
            self.velocity.x *= ground_friction
            self.velocity.z *= ground_friction
    
    def stop(self):
        """Stop all horizontal movement."""
        self.move_direction_x = 0
        self.move_direction_z = 0
        print(f"Agent {self.name} stopped movement")
    
    def __str__(self):
        """Return a string representation of the agent."""
        state = "Grounded" if self.grounded else "Airborne"
        speed = (self.velocity.x**2 + self.velocity.z**2)**0.5
        return (f"{self.name} at {self.position}, "
                f"Speed: {speed:.2f} m/s, "
                f"State: {state}")


# Test the fixed Agent class
def test_fixed_agent():
    # Create world and agent
    world = World(width=50.0, depth=50.0)
    agent = world.add_agent(x=25.0, y=0.0, z=25.0)
    agent.name = "FixedTest"
    
    print(f"Created agent at {agent.position}")
    
    # Test 1: North movement
    print("\nTest 1: Moving North")
    agent.move(0, -1)  # North is negative Z
    
    for i in range(20):
        world.update()
        if i % 5 == 0:
            print(f"Step {i}: {agent}")
    
    # Test 2: East movement
    print("\nTest 2: Moving East")
    agent.move(1, 0)  # East is positive X
    
    for i in range(20):
        world.update()
        if i % 5 == 0:
            print(f"Step {i}: {agent}")
    
    # Test 3: Jump
    print("\nTest 3: Stopping and Jumping")
    agent.stop()
    jump_result = agent.jump()
    
    for i in range(20):
        world.update()
        if i % 5 == 0:
            print(f"Step {i}: {agent}")
    
    print(f"Final position: {agent.position}")

# Run the test
test_fixed_agent()

Created agent at (25.00, 0.00, 25.00)

Test 1: Moving North
Agent FixedTest movement direction set to (0.00, -1.00)
Step 0: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 5: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 10: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 15: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded

Test 2: Moving East
Agent FixedTest movement direction set to (1.00, 0.00)
Step 0: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 5: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 10: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 15: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded

Test 3: Stopping and Jumping
Agent FixedTest stopped movement
Agent FixedTest jumped with force 10.0
Step 0: FixedTest at (25.00, 0.16, 25.00), Speed: 0.00 m/s, State: Airborne
Step 5: FixedTest 